# C07_2CNN (tf.layers)
####  Tensorflow Utitility를 사용한 CNN 구성(tf.layers)


#### CNN(Convolutional Neural Network)은 이미지의 공간 정보를 유지하면서 인접 이미지와의 특징을 효과적으로
#### 인식하고 강조하는 방식으로 이미지의 특징을 추출하는 부분과 이미지를 분류하는 부분으로 구성된다. 
#### 특징 추출 영역은 Filter를 사용하여 공유 파라미터 수를 최소화하면서 이미지의 특징을 찾는 Convolution 레이어와
#### 특징을 강화하고 모으는 Pooling 레이어로 구성된다.
#### CNN은 Filter의 크기, Stride, Padding과 Pooling 크기로 출력 데이터 크기를 조절하고, 
#### 필터의 개수로 출력 데이터의 채널을 결정한다.
#### CNN는 같은 레이어 크기의 Fully Connected Neural Network와 비교해 볼 때, 
#### 학습 파라미터양은 20% 규모다.
#### 은닉층이 깊어질 수록 학습 파라미터의 차이는 더 벌어진다.
#### CNN은 Fully Connected Neural Network와 비교하여 더 작은 학습 파라미터로 더 높은 인식률을 제공한다.

In [3]:
import tensorflow as tf

In [4]:
#Tensorflow에 기본적으로 내장되어 있는 데이터를 로드
from tensorflow.examples.tutorials.mnist import input_data
# 해당 폴더에 내용이 없으면,자동으로 데이터를 다운받기
mnist=input_data.read_data_sets('C:/JupyterSpace/datasets/mnist/data',one_hot=True)

Extracting C:/JupyterSpace/datasets/mnist/data\train-images-idx3-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\train-labels-idx1-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\t10k-images-idx3-ubyte.gz
Extracting C:/JupyterSpace/datasets/mnist/data\t10k-labels-idx1-ubyte.gz


In [10]:
#이미지(28X28) 갯수가 784:X
X=tf.placeholder(tf.float32, [None,28,28, 1])
#0~9 분류:Y
Y=tf.placeholder(tf.float32, [None, 10])

is_training=tf.placeholder(tf.bool)   #변경된 부분

In [12]:
#hidde layer0
#입력값에 가중치를 곱하고 relu함수를 이용해서 레이어 만들기
#padding='SAME': 이미지의 가장 외곽밖에서 한칸 밖으로 움직이는 옵션(Padding)
L1=tf.layers.conv2d(X, 32,[3,3], activation=tf.nn.relu)
L1=tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1=tf.layers.dropout(L1,0.7, is_training)

In [15]:
L2=tf.layers.conv2d(L1, 64,[3,3], activation=tf.nn.relu)
L2=tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2=tf.layers.dropout(L2, 0.7, is_training)

In [20]:
L3=tf.contrib.layers.flatten(L2)
L3=tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3=tf.layers.dropout(L3, 0.5, is_training)

In [21]:
#최종 결과값L3 출력256개를 입력값으로 받아서 0~9 출력값 만들기

model=tf.layers.dense(L3, 10, activation=None)

In [22]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(0.001).minimize(cost)

In [23]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

In [24]:
batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

In [25]:
#epoch:학습 데이터를 전체를 한번 수행하는 것
for epoch in range(15):
    total_cost=0
    
    for i in range(total_batch):
        #지정한 크기만큼 학습데이터를 가져오기
        batch_xs, batch_ys =mnist.train.next_batch(batch_size)
        
        #추가 : 이미지 데이터를 cnn model 형태로 재구성 ([28,28,1])
        batch_xs=batch_xs.reshape(-1,28,28,1)
        _, cost_val=sess.run([optimizer, cost],feed_dict={X:batch_xs, Y:batch_ys, is_training:True})
        total_cost += cost_val
        
    print("Epoch:", '%04d' % (epoch+1), 'Avg.Cost= ','{:.3f}'.format(total_cost/total_batch))
print('최적화 완료')

Epoch: 0001 Avg.Cost=  0.179
Epoch: 0002 Avg.Cost=  0.053
Epoch: 0003 Avg.Cost=  0.035
Epoch: 0004 Avg.Cost=  0.025
Epoch: 0005 Avg.Cost=  0.020
Epoch: 0006 Avg.Cost=  0.015
Epoch: 0007 Avg.Cost=  0.013
Epoch: 0008 Avg.Cost=  0.010
Epoch: 0009 Avg.Cost=  0.008
Epoch: 0010 Avg.Cost=  0.006
Epoch: 0011 Avg.Cost=  0.006
Epoch: 0012 Avg.Cost=  0.007
Epoch: 0013 Avg.Cost=  0.005
Epoch: 0014 Avg.Cost=  0.004
Epoch: 0015 Avg.Cost=  0.005
최적화 완료


In [26]:
#(tf.argmax(model, 1) : model의 행렬을 : 열 순서로 최대값 구하기
is_correct=tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
# cast(): 0과 1 반환
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:',sess.run(accuracy, feed_dict={X:mnist.test.images.reshape(-1,28,28,1), 
                                           Y:mnist.test.labels, is_training:True}))

정확도: 0.9905
